# Summary



### Imports etc.

In [1]:
import copy
import csv
import datetime
import math

import numpy as np
import pandas as pd
import os,sys
import time
print('ready')

ready


In [2]:
import fasttext
import json
import requests
import math
import re
from collections import defaultdict


## Load trained model

In [3]:
date_start = datetime.datetime(2020, 7, 1, 0)
date_end = datetime.datetime(2020, 7, 8, 0)
date_start_str = date_start.strftime('%Y-%m-%d-%H')
date_end_str = date_end.strftime('%Y-%m-%d-%H')
wiki_db = 'wikidata'

size=50
min_count=5

PATH_file = '../output/models/'
# # ## one week
filename = os.path.join(
    PATH_file,
    'embedding-w2v_%s_%s_%s_args-%s-%s.bin'%(
        wiki_db,date_start_str,date_end_str,
        size,min_count
    )
)
model = fasttext.load_model(filename)
print(len(model.get_words()))

2597806


## Define the list of articles

In [4]:
## test: pandemic, disease
list_qid_seed = ['Q81068910','Q84263196']
N_nn = 200
list_wikis = ['enwiki','dewiki','frwiki','eswiki','ruwiki','zhwiki','ptwiki','arwiki','bnwiki','hiwiki']
list_keywords = ['covid','corona','cov'] ## if labels contain these words, remove item
N_max = 10
formatted = False

# ## set1: pandemic, disease
# list_qid_seed = ['Q81068910','Q84263196']
# N_nn = 1000
# list_wikis = ['enwiki','dewiki','frwiki','eswiki','ruwiki','zhwiki','ptwiki','arwiki','bnwiki','hiwiki']
# list_keywords = ['covid','corona','cov'] ## if labels contain these words, remove item
# N_max = 200
# formatted = False

# # set1: xenophobia
# list_qid_seed = ['Q84318312']
# N_nn = 200 ## initial number of nearet neighbors
# list_wikis = ['enwiki']
# list_keywords = []## if labels contain these words, remove item
# N_max=100 ## how many pages to keep from candidates
# formatted = False ## if formatting for wikitext

# # set: gendered impact of the covid pandemic
# list_qid_seed = ['Q89666473']
# N_nn = 200 ## initial number of nearet neighbors
# list_wikis = ['enwiki']
# list_keywords = []## if labels contain these words, remove item
# N_max=100 ## how many pages to keep from candidates
# formatted = False ## if formatting for wikitext

## Query related items and get additional information on articles in respective wikis

#### get nearest neighbors of seed-article and query titles/sitelinks

In [5]:
def getItems(list_qids,list_wikis=None, n_batch=30):
    '''
    for list of qids, get:
    - label
    - description
    - titles (sitelinks)
    if list_wikis is specified, only get titles for specific wikis (e.g. enwiki)
    
    process the qids in batches of n_batches (at most 50, but make smaller to be sure).
    use: https://www.mediawiki.org/wiki/Wikibase/API
    '''
    api_url_base = 'https://wikidata.org/w/api.php'
    ## split into batches
    list_qids_split = np.array_split(list_qids,math.ceil(len(list_qids)/n_batch))
    list_items = []
    i_qid=0    
    for list_qids_batch in list_qids_split:
        
        params = {
            'action':'wbgetentities',
            'props':'sitelinks|labels|descriptions',
            'languages':'en',
            'format' : 'json',
            'ids':'|'.join(list_qids_batch),
        }
        if list_wikis != None:
            params['sitefilter']:'|'.join(list_wikis)

        response = requests.get( api_url_base,params=params)
        result=json.loads(response.text)
        
        for qid in list_qids_batch:
            dict_item = {'qid':qid}

            ## titles via sitelinks
            dict_title = {} ## collect all titles of an item via sitelinks
            titles = result['entities'].get(qid,{}).get('sitelinks',{})
            for wiki, wiki_title in titles.items():
                title = wiki_title['title'].replace(' ','_')
                dict_title[wiki] = title
            dict_item['titles'] = dict_title
            
            ## labels: put '-' if not available (so far only en)
            label = result['entities'].get(qid,{}).get('labels',{}).get('en',{}).get('value','-')
            dict_item['label'] = label
            
            ## description: put '-' if not available (so far only en)
            description = result['entities'].get(qid,{}).get('descriptions',{}).get('en',{}).get('value','-')
            dict_item['description'] = description
            
            list_items += [dict_item]
            
            i_qid+=1    
    return list_items
            

def items_add_titles(list_items, list_wikis = ['enwiki']):
    '''
    
    add information about a list of qids:
    - label
    - description
    - article-titles (sitelinks) for a set of wikis
    '''
    
    list_qids = [h['qid'] for h in list_items]
    list_items_meta = getItems(list_qids, list_wikis=list_wikis)
    
    list_items_new = list_items
    for i_item, item in enumerate(list_items):
        dict_titles = {}
        for wiki in list_wikis:
            title =  list_items_meta[i_item]['titles'].get(wiki,'-')
            dict_titles[wiki] = title
        list_items_new[i_item]['titles'] = dict_titles
        list_items_new[i_item]['label'] = list_items_meta[i_item]['label']
        list_items_new[i_item]['description'] = list_items_meta[i_item]['description']
        
        ## add text
        text = list_items_new[i_item]['label']
        if text == '-':
            text_tmp =  list_items_new[i_item]['description']
            if text_tmp!='-':
                text = text_tmp          

        for wiki in list_wikis:
            if text != '-':
                break
            else:
                text_tmp = list_items_new[i_item]['titles'][wiki]
                if text_tmp!='-':
                    text = text_tmp+' [title-%s]'%(wiki)
        list_items_new[i_item]['text'] = text
    return list_items_new
    

In [6]:
dict_nn_score = defaultdict(list)
for qid in list_qid_seed:
    qid_nn = model.get_nearest_neighbors(qid,k=N_nn)
    for score_tmp,qid_tmp in qid_nn:
        dict_nn_score[qid_tmp] += [score_tmp] 
        
list_nn_qid_tmp = []
list_nn_score_tmp = []
for qid_tmp,score_tmp in dict_nn_score.items():
    list_nn_qid_tmp+=[qid_tmp]
    list_nn_score_tmp += [max(score_tmp)]
    
indsort = np.argsort(list_nn_score_tmp)[::-1][:N_nn]
list_nn_qid = [list_nn_qid_tmp[i] for i in indsort]
list_nn_score = [list_nn_score_tmp[i] for i in indsort]
list_items = [
    {'qid': list_nn_qid[i],
     'score': list_nn_score[i]
    } 
    for i in range(N_nn) 
]
list_items = items_add_titles(list_items,list_wikis = list_wikis)
print(list_items[:2])
print(len(list_items))


[{'qid': 'Q84263196', 'score': 0.9195318818092346, 'titles': {'enwiki': 'Coronavirus_disease_2019', 'dewiki': 'COVID-19'}, 'label': 'COVID-19', 'description': 'zoonotic respiratory syndrome and infectious disease in humans, caused by SARS coronavirus 2', 'text': 'COVID-19'}, {'qid': 'Q81068910', 'score': 0.9195318818092346, 'titles': {'enwiki': 'COVID-19_pandemic', 'dewiki': 'COVID-19-Pandemie'}, 'label': 'COVID-19 pandemic', 'description': 'ongoing pandemic of COVID-19', 'text': 'COVID-19 pandemic'}]
200


#### filter list: disambiguation, keywords

In [7]:
def items_filter_notext(
    list_items):
    '''filter all items that have
    - no English label
    - no description
    - no existing title in any of the selected wikis
    '''
    list_items_new = []
    for item in list_items:
        if item['text'] != '-':
            list_items_new += [item]
    return list_items_new

def items_filter_disambiguation(
    list_items,
    list_keywords=['disambiguation']):
    s = re.compile('|'.join(list_keywords))
    list_items_new = []
    for item in list_items:
        label = item['label']+' '+item['description']
        if s.search(label.lower()) is None:
            list_items_new += [item]
    return list_items_new

def items_filter_keywords(
    list_items,
    list_keywords=[] ):
    if len(list_keywords) == 0:
        return list_items
    else:
        s = re.compile('|'.join(list_keywords))
        list_items_new = []
        for item in list_items:
            label = item['label']
            if s.search(label.lower()) is None:
                list_items_new += [item]
        return list_items_new

In [8]:
## filter if the label contains a keywords as substring
list_items_filtered = list(list_items)
list_items_filtered = items_filter_notext(list_items_filtered)
list_items_filtered = items_filter_disambiguation(list_items_filtered)
list_items_filtered = items_filter_keywords(list_items_filtered,list_keywords = list_keywords)
list_items = list(list_items_filtered)
print(list_items[:2])
print(len(list_items))

[{'qid': 'Q103177', 'score': 0.8773841857910156, 'titles': {'enwiki': 'Severe_acute_respiratory_syndrome', 'dewiki': 'Schweres_akutes_Atemwegssyndrom'}, 'label': 'severe acute respiratory syndrome', 'description': 'viral respiratory disease', 'text': 'severe acute respiratory syndrome'}, {'qid': 'Q83264280', 'score': 0.8562499284744263, 'titles': {'enwiki': 'Huanan_Seafood_Wholesale_Market', 'dewiki': 'Feinkost-Nassmarkt_in_Wuhan'}, 'label': 'Huanan Seafood Wholesale Market', 'description': 'market in Wuhan, Hubei, China', 'text': 'Huanan Seafood Wholesale Market'}]
66


#### get pageviews

In [9]:
def getRedirects(page,project):
    '''
    get all redirects (upto 500)
    for a given page 
    '''
    base_url = 'https://%s.org/w/api.php?action=query&titles=%s&prop=redirects&rdlimit=500&format=json' % (project,page)
    data = [p['title'] for p in list(requests.get(base_url).json()['query']['pages'].values())[0]['redirects']]
    return data
    

def getViews(page,start,end,project):
    """
    get pageviews using this API https://wikitech.wikimedia.org/wiki/Analytics/AQS/Pageviews 
    page: str (article name)
    start: str start date YYYYMMDD (20200101)
    end: str end date YYYYMMDD (20200103)
    project: str, ex: en.wikipedia (project does not include .org)

    """
    base_url = "https://wikimedia.org/api/rest_v1/metrics/pageviews/per-article/%s/all-access/all-agents/%s/daily/%s/%s" % (project,page,start,end)
    try:
        data = requests.get(base_url).json()['items']
        df = pd.DataFrame(data) [['views','timestamp']]
        df.rename(columns={'views':page},inplace=True)
    except KeyError:
        ## no pageviews- we have to set 1 date with 0 counts
        df = pd.DataFrame(columns=[page,'timestamp'],index=[0])
        df.iloc[0,0] = 0
        df.iloc[0,1] = start
    df['timestamp'] = pd.to_datetime(df['timestamp'], format='%Y%m%d%H')
    df.set_index('timestamp',inplace=True)
    return df

def getViewsMultiples(pages,start,end,project):
    """
    Get page views for a list of pages 
    pages: list (list of article's titles) ex: ['Chile','Brasil','Argentina']
    start: str start date YYYYMMDD (20200101)
    end: str end date YYYYMMDD (20200103)
    project: str, ex: en.wikipedia (project does not include .org)
    """
    results = []
    for page in pages:
        try:
            results.append(getViews(page,start,end,project))
        except:
            pass
    return pd.concat(results,axis=1)

def getViewsWithRedirects(page,start,end,project):
    """
    Get all redirects going to 'page' and get pageviews for that article
    page: str (article's title)
    start: str start date YYYYMMDD (20200101)
    end: str end date YYYYMMDD (20200103)
    project: str, ex: en.wikipedia (project does not include .org)
    """
    #start list of page
    pages = [page]
    try:
        #get all redirects to page 
        redirects = getRedirects(page=page,project=project)     
    except:
        #if getRedirects gives an error, we assume that there no pages redirecting to page
        redirects = []
    pages.extend(redirects)
    ## get pages views for all articles
    results = getViewsMultiples(pages=pages,start=start,end=end,project=project)
    #sum all pages views
    results = pd.DataFrame(results.sum(axis=1))
    results.rename(columns={0:page},inplace=True)
    return results      

def items_add_pageviews(list_items,start,end,N_max = 10):
    '''
    for a list of records [{'titles':{'enwiki':'Germany','dewiki':'Deutschland'}}, {}]
    add pageviews
    '''
    
    for i_item,item in enumerate(list_items[:N_max]):
        dict_pageviews = {}
        for wiki in list_wikis:
            project = '%s.wikipedia' % wiki.replace('wiki','')
            page = item['titles'][wiki]
            if page == '-':
                views = np.nan
            else:
                ## get pageviews per day
                views_df = getViewsWithRedirects(page,start,end,project)
                views = int(views_df.sum())
            ## sum pageviews
            dict_pageviews[wiki] = views
        list_items[i_item]['pageviews'] = dict_pageviews
    return list_items[:N_max]

In [13]:
start = str(date_start).replace('-','').split(' ')[0]
end =str(date_end).replace('-','').split(' ')[0]
list_items = items_add_pageviews(list_items,start,end,N_max=N_max)
print(list_items[:2])
print(len(list_items))

[{'qid': 'Q103177', 'score': 0.8773841857910156, 'titles': {'enwiki': 'Severe_acute_respiratory_syndrome', 'dewiki': 'Schweres_akutes_Atemwegssyndrom'}, 'label': 'severe acute respiratory syndrome', 'description': 'viral respiratory disease', 'text': 'severe acute respiratory syndrome', 'pageviews': {'enwiki': 29966, 'dewiki': 2965}}, {'qid': 'Q83264280', 'score': 0.8562499284744263, 'titles': {'enwiki': 'Huanan_Seafood_Wholesale_Market', 'dewiki': 'Feinkost-Nassmarkt_in_Wuhan'}, 'label': 'Huanan Seafood Wholesale Market', 'description': 'market in Wuhan, Hubei, China', 'text': 'Huanan Seafood Wholesale Market', 'pageviews': {'enwiki': 6752, 'dewiki': 122}}]
10


In [14]:
start

'20200701'

#### add topics from wikidata topic model

In [17]:
def getTopic(qid,p_min=0.1):
#     api_url_base = 'https://tools.wmflabs.org/wiki-topic/api/v1/wikidata/topic'
    api_url_base = 'https://wikidata-topic.wmcloud.org/api/v1/topic'
    params = {
        'qid':qid,
         'threshold':0.1
    }
    ## query api: https://tools.wmflabs.org/wiki-topic/
    response = requests.get( api_url_base,params=params)
    result=json.loads(response.text)   
    ## get scores (topic-probabilities) and topics
    scores = [h['score'] for h in result['results']]
    topics = [h['topic'] for h in result['results']]
    ## get topic w maximum probability (if p exceeds p_min)
    ind_max = np.argmax(scores)
    score_max = scores[ind_max]
    if score_max >= p_min:
        topic = topics[ind_max]
    else:
        topic = '-'
    return topic
    
def items_addTopics(list_items):
    for i_item,item in enumerate(list_items):
        qid = item['qid']
        topic = getTopic(qid)
        list_items[i_item]['topic'] = topic
    return list_items

In [18]:
list_items = items_addTopics(list_items)
print(list_items[:2])
print(len(list_items))

[{'qid': 'Q103177', 'score': 0.8773841857910156, 'titles': {'enwiki': 'Severe_acute_respiratory_syndrome', 'dewiki': 'Schweres_akutes_Atemwegssyndrom'}, 'label': 'severe acute respiratory syndrome', 'description': 'viral respiratory disease', 'text': 'severe acute respiratory syndrome', 'pageviews': {'enwiki': 29966, 'dewiki': 2965}, 'topic': 'STEM.STEM*'}, {'qid': 'Q83264280', 'score': 0.8562499284744263, 'titles': {'enwiki': 'Huanan_Seafood_Wholesale_Market', 'dewiki': 'Feinkost-Nassmarkt_in_Wuhan'}, 'label': 'Huanan Seafood Wholesale Market', 'description': 'market in Wuhan, Hubei, China', 'text': 'Huanan Seafood Wholesale Market', 'pageviews': {'enwiki': 6752, 'dewiki': 122}, 'topic': 'History_and_Society.Society'}]
10


## Fromat and save data

In [19]:
df = pd.DataFrame()
if formatted == True:
    df['Wikidata-item'] = [ '[[wikidata:{0}|{0}]]'.format(h['qid']) for h in list_items ]
else:
    df['Wikidata-item'] = [ '{0}'.format(h['qid']) for h in list_items ]
df['label'] = [ h['text'] for h in list_items ]
df['score'] = [ '%.3f'%(h['score']) for h in list_items ]
df['topic'] = [ h['topic'] for h in list_items ]
for wiki in list_wikis:
    if formatted == True:
        df['%s:title'%(wiki)] = [  '[[%s:%s|%s]]'%(wiki[:-4],h['titles'][wiki],h['titles'][wiki].replace('_',' ')) if h['titles'][wiki]!='-' else '-'  for h in list_items ]
    else:
        df['%s:title'%(wiki)] = [  '%s'%(h['titles'][wiki].replace('_',' ')) if h['titles'][wiki]!='-' else '-'  for h in list_items ]

    df['%s:views'%wiki] = [ '%s'%(h['pageviews'][wiki]) if (h['pageviews'][wiki])>=500 else '<500' if np.isnan((h['pageviews'][wiki]))==False else '-'   for h in list_items ]

df.index = np.arange(1, len(df) + 1)
df.head()

,Wikidata-item,label,score,topic,enwiki:title,enwiki:views,dewiki:title,dewiki:views
1,Q103177,severe acute respiratory syndrome,0.877,STEM.STEM*,Severe acute respiratory syndrome,29966,Schweres akutes Atemwegssyndrom,2965
2,Q83264280,Huanan Seafood Wholesale Market,0.856,History_and_Society.Society,Huanan Seafood Wholesale Market,6752,Feinkost-Nassmarkt in Wuhan,<500
3,Q834456,2002–2004 SARS outbreak,0.841,STEM.STEM*,2002–2004 SARS outbreak,28534,SARS-Pandemie 2002/2003,<500
4,Q101452,2009 swine flu pandemic,0.838,STEM.STEM*,2009 swine flu pandemic,74783,Pandemie H1N1 2009/10,<500
5,Q36855,swine influenza,0.833,STEM.STEM*,Swine influenza,28158,Schweineinfluenza,1265


In [26]:
filename_save = os.path.join(
    os.pardir,
    'output',
    'lists',
    'reading-sessions-related-articles_%s_N%s_%s--%s_formatted-%s.csv'%(
        '-'.join(list_qid_seed),
        N_max,
        str(date_start_str), str(date_end_str), formatted
    )
)
df.to_csv(filename_save)

In [27]:
print('success')

success
